In [8]:
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
from scipy.stats import gmean
import os
import statsmodels.api as sm
from pandas.tseries.offsets import MonthEnd
import warnings 
warnings.filterwarnings('always')
from sklearn.metrics import r2_score, mean_squared_error


# The code is written with the help of OpenAI's Chatgpt.com.

# Data Prep

In [2]:
# Set path and file names
folder_path = './data/'
file_list = [
    'spx_totret_1995_monthly.xlsx',         # US Equities
    'UST30Y_totret_1995_monthly.xlsx',   # US Government Bond
    'HY_totret_1995_monthly.xlsx',    # US HY Corporate Bonds
    'gsci_totret_1995_monthly.xlsx', # Global Commodities
    'EURHY_data.xlsx', # EUR Corporate Bonds
    'EUREQ_data.xlsx', # EUR Equities
    'FXY_data.xlsx', # USDJPY ETF Total Return
     'Gold_data.xlsx', # Gold

]

sheet_name = 'Worksheet'
col_name = 'TOT_RETURN_INDEX_GROSS_DVDS'
start_date = pd.to_datetime('2010-01-01')

# Initialize container for all standardized returns
standardized_df = pd.DataFrame()

# Loop over files
for file in file_list:
    filepath = os.path.join(folder_path, file)
    
    # Extract asset name from filename
    asset_name = file.split('_')[0].upper()  # 'spx_totret_1995_monthly.xlsx' → 'SPX'

    # Load and clean data
    df = pd.read_excel(filepath, sheet_name=sheet_name)
    df = df.rename(columns={col_name: 'Total_Return_Index'}).dropna()
    df = df[df['Date'] >= start_date].copy()
    df = df.set_index('Date')

    # Compute monthly returns
    df['Return'] = df['Total_Return_Index'].pct_change()
    df = df.dropna()

    # Compute mean and volatility
    monthly_returns = df['Return']
    mean_return = gmean(1 + monthly_returns) - 1
    vol_return = monthly_returns.std()

    # Standardize return
    df[f'{asset_name}'] = (monthly_returns - mean_return) / vol_return

    # Add to master DataFrame
    standardized_df = pd.concat([standardized_df, df[[f'{asset_name}']]], axis=1).dropna()

# Reset index to have 'Date' as column
standardized_df = standardized_df.reset_index()
standardized_df_1 = standardized_df
display(standardized_df_1.head())
display(standardized_df_1.tail())

,Date,SPX,UST30Y,HY,GSCI,EURHY,EUREQ,FXY,GOLD
0,2010-03-31,1.184608,-0.530763,1.277132,0.397559,-0.397344,1.062714,-1.739296,-1.106774
1,2010-04-30,0.122757,0.751679,0.892005,0.613177,2.199185,2.424337,-0.066530,-0.530417
2,2010-06-30,-1.500856,1.276440,0.357045,0.226211,-1.672478,-1.920082,1.157514,1.318014
3,2010-08-31,-1.329079,1.959475,-0.231386,-0.836337,-1.159306,-1.795480,1.144543,-0.884953
4,2010-09-30,1.873141,-0.658468,1.217676,1.565948,-0.644847,-0.866383,0.338207,-0.661408


,Date,SPX,UST30Y,HY,GSCI,EURHY,EUREQ,FXY,GOLD
124,2024-10-31,-0.469507,-1.289861,-0.512055,0.027249,-0.027333,0.617184,-1.902411,0.843801
125,2024-12-31,-0.821469,-1.450095,-0.456465,0.423074,-0.062569,-1.291312,-1.730543,-1.284009
126,2025-01-31,0.410162,-0.016282,0.416141,0.361181,-0.252903,-0.443162,0.640791,-0.269920
127,2025-02-28,-0.564339,1.250348,0.077880,-0.263806,0.643485,0.082555,1.252147,-1.789744
128,2025-03-31,-1.596070,-0.366401,-0.746498,0.426099,0.074357,-0.547447,0.215081,-1.015440


In [3]:
# Set path and file names
folder_path = './data/'
file_list = [
    'EEM_monthly_prices.xlsx', # Emerging Market Equities
    'VNQ_monthly_prices.xlsx', # US Real estate ETF
    'IGF_monthly_prices.xlsx', # Global Infrastructure ETF
     'unemp_data.xlsx', # US Unemployment
     'CPI_data.xlsx', # US Inflation (CPI)
]

sheet_name = 'Worksheet'
col_name = 'TOT_RETURN_INDEX_GROSS_DVDS'
start_date = pd.to_datetime('2010-01-01')

# Initialize container for all standardized returns
standardized_df = pd.DataFrame()

# Loop over files
for file in file_list:
    filepath = os.path.join(folder_path, file)
    
    # Extract asset name from filename
    asset_name = file.split('_')[0].upper()  # 'spx_totret_1995_monthly.xlsx' → 'SPX'

    # Load and clean data
    df = pd.read_excel(filepath, sheet_name=sheet_name)
    df = df.rename(columns={col_name: 'Total_Return_Index'}).dropna()
    df = df[df['Date'] >= start_date].copy()
    df = df.set_index('Date')

    # Compute monthly returns
    df['Return'] = df['Total_Return_Index'].pct_change()
    df = df.dropna()

    # Compute mean and volatility
    monthly_returns = df['Return']
    mean_return = gmean(1 + monthly_returns) - 1
    vol_return = monthly_returns.std()

    # Standardize return
    df[f'{asset_name}'] = (monthly_returns - mean_return) / vol_return

    # Add to master DataFrame
    standardized_df = pd.concat([standardized_df, df[[f'{asset_name}']]], axis=1).dropna()

# Reset index to have 'Date' as column
standardized_df_2 = standardized_df.reset_index()
standardized_df_2['Date'] = standardized_df_2['Date'] + MonthEnd()
display(standardized_df_2.head())
display(standardized_df_2.tail())

,Date,EEM,VNQ,IGF,UNEMP,CPI
0,2010-04-30,-0.074012,1.400736,-0.424852,0.025688,-0.714625
1,2010-05-31,-1.829766,-1.150317,-2.500427,-0.142828,-0.998445
2,2010-06-30,-0.431933,-1.297221,-0.806843,-0.090167,-0.960223
3,2010-07-31,2.177581,1.914237,2.759368,0.025688,-0.094147
4,2010-08-31,-0.658261,-0.371159,-0.551676,0.084849,-0.248542


,Date,EEM,VNQ,IGF,UNEMP,CPI
176,2024-12-31,-0.675763,-1.901995,-1.503414,-0.106718,0.578661
177,2025-01-31,0.695247,0.379335,0.640709,-0.109947,0.965750
178,2025-02-28,0.175860,0.583960,-0.005591,0.164715,0.015639
179,2025-03-31,0.173387,-0.813164,0.379334,0.161324,-0.991094
180,2025-04-30,-0.016264,-0.402441,0.581431,0.025688,0.034419


In [4]:
# Merge Dataframe
merged_df = pd.merge(standardized_df_1, standardized_df_2, on='Date', how='inner')
display(merged_df.head())
display(merged_df.tail())

,Date,SPX,UST30Y,HY,GSCI,EURHY,EUREQ,FXY,GOLD,EEM,VNQ,IGF,UNEMP,CPI
0,2010-04-30,0.122757,0.751679,0.892005,0.613177,2.199185,2.424337,-0.066530,-0.530417,-0.074012,1.400736,-0.424852,0.025688,-0.714625
1,2010-06-30,-1.500856,1.276440,0.357045,0.226211,-1.672478,-1.920082,1.157514,1.318014,-0.431933,-1.297221,-0.806843,-0.090167,-0.960223
2,2010-08-31,-1.329079,1.959475,-0.231386,-0.836337,-1.159306,-1.795480,1.144543,-0.884953,-0.658261,-0.371159,-0.551676,0.084849,-0.248542
3,2010-09-30,1.873141,-0.658468,1.217676,1.565948,-0.644847,-0.866383,0.338207,-0.661408,2.194769,0.553082,1.516725,0.025688,-0.190293
4,2010-11-30,-0.250416,-0.487142,-0.817480,0.333811,-0.712199,-1.170027,-1.319898,-0.400942,-0.595230,-0.481689,-1.337143,0.262329,0.157414


,Date,SPX,UST30Y,HY,GSCI,EURHY,EUREQ,FXY,GOLD,EEM,VNQ,IGF,UNEMP,CPI
123,2024-10-31,-0.469507,-1.289861,-0.512055,0.027249,-0.027333,0.617184,-1.902411,0.843801,-0.627407,-0.614365,-0.411725,0.025688,0.055486
124,2024-12-31,-0.821469,-1.450095,-0.456465,0.423074,-0.062569,-1.291312,-1.730543,-1.284009,-0.675763,-1.901995,-1.503414,-0.106718,0.578661
125,2025-01-31,0.410162,-0.016282,0.416141,0.361181,-0.252903,-0.443162,0.640791,-0.269920,0.695247,0.379335,0.640709,-0.109947,0.965750
126,2025-02-28,-0.564339,1.250348,0.077880,-0.263806,0.643485,0.082555,1.252147,-1.789744,0.175860,0.583960,-0.005591,0.164715,0.015639
127,2025-03-31,-1.596070,-0.366401,-0.746498,0.426099,0.074357,-0.547447,0.215081,-1.015440,0.173387,-0.813164,0.379334,0.161324,-0.991094


##### Macro Factor DataFrame

### Regression Analysis
- From 2010-2024, empirically verify if the macro data series are relevant for the asset 
price series. You can do this via linear regression, as discussed in class, or via some 
other method. Explain and justify your approach.  

In [6]:
def time_series_regression(portfolio, factors, resid = False,start = '2010-01-01',end = '2024-12-31', intercept = False):
    
    ff_report = pd.DataFrame(index=portfolio.columns)
    bm_residuals = pd.DataFrame(columns=portfolio.columns)
    factors_reg = factors[(factors.index >= start) & (factors.index <= end)]
    portfolio_reg = portfolio[(portfolio.index >= start) & (portfolio.index <= end)]

    if intercept:
        X = sm.add_constant(factors)

    for portf in portfolio_reg.columns:
        y = portfolio_reg[portf]
        if not intercept:
            res = sm.OLS(y, factors_reg, missing='drop').fit()

            ff_report.loc[portf, 'beta_gold'] = res.params.iloc[0]
            ff_report.loc[portf, 'beta_cpi'] = res.params.iloc[1]
            ff_report.loc[portf, 'beta_unemp'] = res.params.iloc[2]
        else:
            res = sm.OLS(y, X, missing='drop').fit()

            ff_report.loc[portf, 'constant'] = res.params.loc['const']
            ff_report.loc[portf, 'beta_gold'] = res.params.iloc[0]
            ff_report.loc[portf, 'beta_cpi'] = res.params.iloc[1]
            ff_report.loc[portf, 'beta_unemp'] = res.params.iloc[2]
            

        if resid:
            bm_residuals[portf] = res.resid # returns a Data Frame of residuals 

    if resid: # returns a Data Frame of residuals 
        return bm_residuals
        
    return ff_report

In [44]:
macro_factors = merged_df[['GOLD','CPI','UNEMP','Date']].set_index('Date')
assets = merged_df.drop(columns = ['GOLD','CPI','UNEMP']).set_index('Date')
time_series_regression = time_series_regression(assets, macro_factors, intercept = False)
display(time_series_regression)

,beta_gold,beta_cpi,beta_unemp
SPX,0.010160,0.079394,0.186411
UST30Y,-0.096065,-0.240880,-0.041710
HY,-0.002577,0.016904,0.091766
GSCI,0.065035,0.341934,0.061267
EURHY,0.177027,0.137664,-0.050167
EUREQ,0.278103,0.064784,-0.051801
FXY,-0.080475,-0.175266,-0.046094
EEM,0.007020,0.038555,0.079444
VNQ,-0.062745,-0.009838,0.084410
IGF,-0.030851,0.081587,0.095638


In [ ]:
# y-values
asset_returns = pd.DataFrame({
    'SPX': merged_df['SPX'],
    'UST30Y': merged_df['UST30Y'],
    'HY': merged_df['HY'],
    'GSCI': merged_df['GSCI'],
    'EURHY': merged_df['EURHY'],
    'EUREQ': merged_df['EUREQ'],
    'FXY': merged_df['FXY'],
    'EEM': merged_df['EEM'],
    'VNQ': merged_df['VNQ'],
    'IGF': merged_df['IGF'],
})

# x-values
x_values_list = ['GOLD', 'UNEMP', 'CPI']

for macro_asset in x_values_list:
    macro_data = pd.DataFrame({
        f'{macro_asset}': merged_df[f'{macro_asset}'],
    })

    data = pd.merge(asset_returns, macro_data, left_index=True, right_index=True, how='inner')

    X = data[macro_data.columns]
    Y = data[asset_returns.columns]

    results = []

    for asset in Y.columns:
        y = Y[asset]
        model = sm.OLS(y, X).fit()
        r_squared = model.rsquared * 100 # Show in Percent
        rsquared_adj = model.rsquared_adj * 100 # Show in Percent
        p_val = model.pvalues.iloc[0] * 100 # Show in Percent
        
        results.append({
            'Asset': asset,
            'R-Squared (%)': r_squared,
            'Coefficients': model.params.iloc[0],
            'P-Value (%)': p_val,
            'Adjusted R-Squared (%)': rsquared_adj,
            'F-Statistic': model.fvalue,
            'MSE': mean_squared_error(y, model.fittedvalues)
        })
    regression_results = pd.DataFrame(results)

    regression_results = regression_results.sort_values(by='R-Squared (%)', ascending=False)
    print(f"Regression of asset returns (y) on {macro_asset} (x):")
    display(regression_results)

Regression of asset returns (y) on GOLD (x):


,Asset,R-Squared (%),Coefficients,P-Value (%),Adjusted R-Squared (%),F-Statistic,MSE
5,EUREQ,8.131834,0.277205,0.105438,7.408462,11.241575,0.836437
4,EURHY,3.534502,0.166773,3.287719,2.774931,4.653287,0.731394
1,UST30Y,1.172589,-0.107802,22.188980,0.394420,1.506857,0.943712
6,FXY,0.915085,-0.097876,28.085984,0.134889,1.172891,0.999428
8,VNQ,0.416004,-0.069360,46.772388,-0.368121,0.530533,1.109604
3,GSCI,0.348275,0.062144,50.647620,-0.436385,0.443855,1.064683
9,IGF,0.139616,-0.040084,67.419056,-0.646686,0.177560,1.107260
0,SPX,0.029661,0.017812,84.639657,-0.757507,0.037680,1.030312
2,HY,0.002019,-0.004775,95.969144,-0.785366,0.002564,1.088103
7,EEM,0.001271,-0.003964,96.801223,-0.786120,0.001614,1.190703


Regression of asset returns (y) on UNEMP (x):


,Asset,R-Squared (%),Coefficients,P-Value (%),Adjusted R-Squared (%),F-Statistic,MSE
0,SPX,3.430747,0.158341,3.559978,2.670360,4.511839,0.995260
4,EURHY,1.932766,-0.101936,11.611766,1.160583,2.502990,0.743539
5,EUREQ,1.152402,-0.086255,22.593503,0.374075,1.480613,0.899983
8,VNQ,1.029214,0.090176,25.262710,0.249917,1.320695,1.102771
2,HY,0.950118,0.085620,27.179625,0.170198,1.218225,1.077787
9,IGF,0.642086,0.071052,36.668543,-0.140259,0.820720,1.101688
7,EEM,0.526433,0.066669,41.385346,-0.256823,0.672108,1.184450
3,GSCI,0.343987,-0.051049,50.910946,-0.440706,0.438371,1.064729
1,UST30Y,0.244041,0.040650,57.823768,-0.541439,0.310691,0.952579
6,FXY,0.030223,0.014702,84.496666,-0.756941,0.038394,1.008353


Regression of asset returns (y) on CPI (x):


,Asset,R-Squared (%),Coefficients,P-Value (%),Adjusted R-Squared (%),F-Statistic,MSE
3,GSCI,9.494596,0.310964,0.038159,8.781955,13.323112,0.966963
1,UST30Y,5.217683,-0.217933,0.922382,4.471365,6.991238,0.905085
4,EURHY,3.407669,0.156936,3.623639,2.647100,4.480418,0.732356
6,FXY,2.363969,-0.150764,8.192053,1.595181,3.074931,0.984814
5,EUREQ,0.935511,0.090108,27.553033,0.155476,1.199319,0.901958
9,IGF,0.158165,0.040887,65.452860,-0.627992,0.201187,1.107054
8,VNQ,0.134496,-0.037796,67.988688,-0.651847,0.171040,1.112741
2,HY,0.019401,-0.014186,87.550760,-0.767848,0.024644,1.087914
0,SPX,0.017589,0.013145,88.141776,-0.769674,0.022342,1.030436
7,EEM,0.005328,0.007777,93.456541,-0.782031,0.006768,1.190655


### 5. 
- Should your estimation sample period be different? Why or why not? If it should be 
different, adjust it and do your analysis on the altered sample. 